In [69]:
%load_ext autoreload
%autoreload 2

import ee
# from folium import Map
import utils
# Authenticate to Earth Engine
ee.Authenticate()
# Initialize Earth Engine instance
ee.Initialize(project="reeftruth")
#!!! (for readme)
# https://stackoverflow.com/questions/78374548/no-authentication-box-appears-when-authenticating-google-earth-engine-gee-pyth


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define functions

In [70]:
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = utils.add_ee_layer

### Set visualisation parameters

In [77]:
protectedareas_vps = {
  "palette": ["2ed033"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.05
}
gdcr_vps = {
  "palette": ["green"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8
}
wri_vps = {
  "palette": ["red"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.4
}
reefcheck_vps = {
  "color": "white",  
  "pointSize": 200   
}

copernicus_vps = {
  "min": 0,
  "max": 2,
  "gamma": 1.5
}

# centring and zoom
lat = -20
lon = 150
zoom = 10

## Add reef layers to map

TODO: detail layers

In [78]:

# initialise map with bacgkround image
reef_map = folium.Map(location=[lat, lon], zoom_start=zoom)

### COPERNICUS
# Load Copernicus image collection and filter by date
copernicus = ee.ImageCollection("COPERNICUS/S3/OLCI").filterDate("2018-02-02", "2018-02-03")

# Select bands for visualization and apply band-specific scale factors.
copernicus_image = copernicus.select(["Oa08_radiance", "Oa06_radiance", "Oa04_radiance"]) \
                              .median() \
                              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))

reef_map.add_ee_layer(copernicus_image, copernicus_vps, "copernicus background")

### ALLEN CORAL ATLAS
aca_ds = ee.Image("ACA/reef_habitat/v2_0") 

# Example mask application
aca_reef_extent = aca_ds.select("reef_mask").selfMask()
reef_map.add_ee_layer(aca_reef_extent, {"palette": ["EEEEEE"]}, "ACA Global reef extent")

# Geomorphic zonation classification
aca_geomorphic_zonation = aca_ds.select("geomorphic").selfMask()
reef_map.add_ee_layer(aca_geomorphic_zonation, {}, "ACA Geomorphic zonation")

# Benthic habitat classification
aca_benthic_habitat = aca_ds.select("benthic").selfMask()
reef_map.add_ee_layer(aca_benthic_habitat, {"palette": ["FF7F50"]}, "ACA Benthic habitat")


### UNEP-WCMC World Database of Coral Reefs (GDCR)
unep_gdcr = ee.FeatureCollection("projects/reeftruth/assets/WCMC008_CoralReef2021_Py_v4_1") 
gdcr_image = ee.Image().float().paint(unep_gdcr, "GDCR")
reef_map.add_ee_layer(gdcr_image, gdcr_vps, "UNEP-GDCR_polygons")

### World Resources Institute (WRI) Reefs at Risk
wri_500 = ee.FeatureCollection("projects/reeftruth/assets/reef_500_poly") 
wri_image = ee.Image().float().paint(wri_500, "WRI_500")
reef_map.add_ee_layer(wri_image, wri_vps, "WRI_500_polygons")

### Reef Check
# # reef_check = ee.FeatureCollection("projects/...") 
# # reef_map.add_child(reef_check, reefcheck_vps, "Reef Check points")

### World Database on Protected Areas (WDPA)
wdpa_ds = ee.FeatureCollection("WCMC/WDPA/current/polygons")
wdpa_image = ee.Image().float().paint(wdpa_ds, "REP_AREA")
reef_map.add_ee_layer(wdpa_image, protectedareas_vps, "protected areas")

reef_map.add_child(folium.LayerControl())
display(reef_map)